# Representação de Trajetória
O objetivo deste notebook é apresentar os resultados obtidos de maneira visual.

In [1]:
import data_processing
import re

data = data_processing.load_student_data()
disciplinas = data_processing.load_disciplinas()

CSD20
['ICSD20', 'IF61B']
CSF13
['ICSF13', 'IF61C']
FI71S
['FIS7F1', 'FI71M', 'FI71Z']
MA71A
['MAT7C1', 'MA61A', 'MA71Z']
MA71B
['MAT7AL', 'MAT7GA', 'MA61B', 'MA71I', 'MA71Y', 'MA72I']
CSE20
['ICSE20', 'IF62C']
CSF20
['ICSF20', 'IF62E']
EEX11
['ELEX10', 'ELE11']
FI72N
['FIS7E1', 'FI61A', 'FI62A', 'FI71A', 'FI71Y', 'FI72A', 'FI72B']
FI72S
['FIS7F2', 'FI72M', 'FI72Z']
MA72A
['MA62A', 'MA72H', 'MA75D']
QB70C
['QBI7QE', 'QBI7QT', 'QB64A', 'QB70D', 'QB70I']
CSF30
['ICSF30', 'IF63C']
EEB21
['ELEB20', 'ELP21', 'EL63A', 'EL72F']
EEX21
['ELEX20', 'ELE41', 'ELX91']
FI73S
['FIS7F3', 'FI73M']
MA70G
['FI74K', 'MAT7ED', 'MA63B', 'MA70B', 'MA70Z']
MA70H
['EST70A', 'MA65A', 'MA70F']
CSD21
['ICSD21', 'IF63E']
EEB22
['ELEB21', 'ELP32', 'EL64F', 'EL73F']
EEB31
['ELEB30', 'ELF41', 'EL65A', 'EL75F']
EEF11
['ELB13', 'ELEF10', 'EL62A']
EEQ31
['ELB66', 'ELEQ30', 'EL65D']
FI74S
['FIS7F4', 'FI74M']
CSA30
['ICSA30', 'IF64C']
CSB30
['ICSB30', 'IF65E']
CSG20
['ICSG20', 'IF65D']
CSW30
['ELEW30', 'ELF61', 'EL66C']
E

In [2]:
for item in disciplinas:
    if re.search('P[1-9]', item):
        print(item)

In [3]:
sugestoes = []
with open('sugestoes.txt') as file:
    for line in file:
        nome = line.strip().split('\n')[0]
        if nome in disciplinas:
            disc = disciplinas[nome]
            sugestoes.append(disc)
            if disc.trilha == 'Optativas Isoladas' or disc.trilha == 'obrigatoria':
                print(f'{disc.nome} não está em nenhuma trilha?')
        else:
            print(f'{nome} não está em disciplinas')

QB7AH não está em nenhuma trilha?
ED70U não está em nenhuma trilha?
FCH7HB não está em nenhuma trilha?
EL64B não está em nenhuma trilha?
ED70T não está em nenhuma trilha?
EL66B não está em nenhuma trilha?
MA7AV não está em nenhuma trilha?
CSG48 não está em nenhuma trilha?
CSG42 não está em nenhuma trilha?
CSG44 não está em nenhuma trilha?
ES70N não está em nenhuma trilha?
ES70J não está em nenhuma trilha?
IF6CK não está em nenhuma trilha?
GE70F não está em nenhuma trilha?
EL65B não está em nenhuma trilha?
EL6AE não está em nenhuma trilha?
IF6BV não está em nenhuma trilha?
ES70B não está em nenhuma trilha?
EL6CB não está em nenhuma trilha?
EL6EE não está em nenhuma trilha?
EL6CC não está em nenhuma trilha?
EL65F não está em nenhuma trilha?
EL6BC não está em nenhuma trilha?
EL6CA não está em nenhuma trilha?
IF6CJ não está em nenhuma trilha?
EL6BB não está em nenhuma trilha?
EL6BA não está em nenhuma trilha?


In [4]:
for item in sugestoes:
    print(item.equivalentes)

['ELTA4', 'EL6DC']
['QB56F', 'QB60C', 'QB70L']
[]
[]
[]
[]
['ICSHX1', 'IF66H']
[]
['ELTA3', 'EL6DB']
[]
['ICSM46', 'IF6AG']
['ELTE2', 'EL6AA', 'EL68G', 'ICSD41', 'IF6BO']
['ED60G', 'ED70H']
[]
[]
[]
[]
['ICSR41', 'IF67B']
['ELEW40', 'ELN7AB', 'ELTE1']
['ICSI52', 'IF68D']
[]
[]
[]
['ICSR44', 'IF68E']
['ICSH31', 'IF63F', 'IF63O']
[]
['ICSE43', 'IF6BT']
['ICSH43', 'IF68B']
[]
[]
['FI65C', 'FI67A', 'ME66F']
['ELTA1', 'EL68D']
['CC73A', 'COC73F', 'FI62B']
['ICSI55', 'IF6BQ']
['ICSI51', 'IF6AC']
['ICSV41', 'IF6BJ']
[]
['ICSB51', 'IF6AH']
['ICSV30', 'IF6BM', 'IF69D']
['ICSB41', 'IF66E']
['ICSV40', 'IF6AF']
['ELB52', 'ET64C', 'ET74E']
['ICSA48', 'IF6AV']
[]
[]
[]
[]
[]
[]
['ICSR43', 'IF6BP']
[]
['ELTA2', 'EL6DA']
['ICSR42', 'IF6AP']
[]
['EDU70J', 'ED60F', 'ED70G', 'ED70V']
['ICSE40', 'IF66G']
[]
[]
['ICSI56', 'IF6AK']
[]
[]
[]
['ICSD42', 'IF6AQ']
['ICSI41', 'IF6AJ']
[]
[]
[]
[]
['ICSM31', 'IF66F']
['ICSD45', 'IF67F']
['MAT7N1', 'MA63C']
['ICSA47', 'IF6BB']
[]
[]
['GE60J', 'ICSG42']
['ICSG44', 

In [5]:
from typing import List

def dar_sugestoes(CHS_desejada : int, sugestoes : List, id_aluno : int, dados = None):
    # Carga horária desejada
    CH_desejada = CHS_desejada * 15
    if len(dados) == 0:
        dados_aluno = (data['todos'][data['todos']['ID_ANONIMO'] == id_aluno])
    else:
        dados_aluno = (dados[dados['ID_ANONIMO'] == id_aluno])

    dados_aluno = dados_aluno[dados_aluno['NOTA'] >= 6]
    disciplinas_cursadas = dados_aluno['CODIGO'].unique()
    it = 0

    for disc in disciplinas:
        i = 0
        for item in disciplinas_cursadas:
            if item in disciplinas[disc].equivalentes:
                disciplinas_cursadas[i] = disc
                break
            i += 1

    sugest_aluno = sugestoes.copy()

    trilha_1 = None
    trilha_2 = None

    obrigatorias_por_periodo = []
    ch_trilha1 = 0
    ch_trilha2 = 0
    ch_isoladas = 0

    for i in range(0,12):
        obrigatorias_por_periodo.append([])

    for disc in disciplinas:
        try:
            if disciplinas[disc].trilha == 'obrigatoria' and disc not in disciplinas_cursadas and disc not in ["CSX53"]:
                #print(disc)
                obrigatorias_por_periodo[disciplinas[disc].periodo - 1].append(disc)
                #print(disciplinas[disc].codigo)
            elif disc in disciplinas_cursadas and disciplinas[disc].trilha != 'obrigatoria':
                if ((trilha_1 == None and disciplinas[disc].trilha != 'Optativas Isoladas') or disciplinas[disc].trilha == trilha_1):
                    trilha_1 = disciplinas[disc].trilha
                    ch_trilha1 += disciplinas[disc].ch
                    #print(f'trilha 1 : {trilha_1}')
                elif ((trilha_2 == None and disciplinas[disc].trilha != trilha_1 and disciplinas[disc].trilha != 'Optativas Isoladas') or disciplinas[disc].trilha == trilha_2): 
                    trilha_2 = disciplinas[disc].trilha
                    #print(f'trilha 2 : {trilha_2}')
                    ch_trilha2 += disciplinas[disc].ch
                else:
                    #print(f'{disciplinas[disc].codigo}, {disciplinas[disc].trilha}')
                    ch_isoladas += disciplinas[disc].ch
        except KeyError:
            print(disc)
    # Inicio algoritmo guloso
    formado = False
    periodo_atual = 0
    semestre_atual = 0
    ch_atual = 0
    disciplinas_semestre = []

    while not formado:
        # Processamento por período
        disciplinas_semestre.append([])
        ch_atual = 0

        tries_periodo = 0
        while ch_atual < CH_desejada and not formado and it < 10 and tries_periodo < 200:
            # Priorizamos sempre colocar as obrigatórias antes
            while periodo_atual < 11:
                #print(f'periodo atual {periodo_atual}')
                if len(obrigatorias_por_periodo[periodo_atual]) > 0:
                    for disc in obrigatorias_por_periodo[periodo_atual]:
                        disc = disciplinas[disc]
                        #print(disc.codigo)
                        #print(disc.nome)
                        ch_atual += disc.ch
                        disciplinas_semestre[semestre_atual].append(disc.codigo)
                        obrigatorias_por_periodo[periodo_atual].remove(disc.codigo)
                        if len(obrigatorias_por_periodo[periodo_atual]) == 0:
                            periodo_atual += 1
                        if ch_atual > CH_desejada:
                            break
                    break
                else:
                    periodo_atual += 1
            tries_periodo += 1
            if ch_atual > CH_desejada:
                break

            # Depois tentamos encaixar as optativas, priorizando as trilhas escolhidas
            try:
                for disc in sugest_aluno:
                    if (trilha_1 == None or disciplinas[disc].trilha == trilha_1)  and ch_trilha1 < 90 and disciplinas[disc].trilha != 'Optativas Isoladas':
                        #print(f'trilha 1 : {trilha_1}')
                        trilha_1 = disciplinas[disc].trilha
                        disciplinas_semestre[semestre_atual].append(disc)
                        sugest_aluno.remove(disc)
                        ch_trilha1 += disciplinas[disc].ch
                        ch_atual += disciplinas[disc].ch
                    elif (trilha_2 == None or disciplinas[disc].trilha == trilha_2) and ch_trilha2 < 90 and disciplinas[disc].trilha != 'Optativas Isoladas':
                        #print(f'trilha 2 : {trilha_2}')
                        trilha_2 = disciplinas[disc].trilha
                        disciplinas_semestre[semestre_atual].append(disc)
                        sugest_aluno.remove(disc)
                        ch_trilha2 += disciplinas[disc].ch
                        ch_atual += disciplinas[disc].ch
                    elif ch_isoladas < 90 and disciplinas[disc].trilha != trilha_1 and disciplinas[disc].trilha != trilha_2:
                        disciplinas_semestre[semestre_atual].append(disc)
                        sugest_aluno.remove(disc)
                        ch_isoladas += disciplinas[disc].ch
                        ch_atual += disciplinas[disc].ch

                    if ch_atual > CH_desejada:
                        break
            except KeyError:
                print(disc)
            formado = True
            for periodo in obrigatorias_por_periodo:
                if (len(periodo) > 0 and tries_periodo < 200):
                    #print(f'Obrigatoria faltando: {len(periodo)} {periodo}')
                    formado = False
            if ch_trilha1 < 90 or ch_trilha2 < 90 or ch_isoladas < 90:
                #print(f'CH T1 - {ch_trilha1} T2 {ch_trilha2} T Isolada {ch_isoladas}')
                formado = False
                
            #print(formado)
        
        #print(formado)

        semestre_atual += 1

        if formado == False:
            it += 1
        

        if formado == True or it > 15:
            #print(f'Obrigatoria faltando: {len(periodo)} {periodo}')
            #print(f'CH T1 - {ch_trilha1} T2 {ch_trilha2} T Isolada {ch_isoladas}')
            return disciplinas_semestre



In [6]:
disciplinas['EEC31'].periodo

8

In [7]:
#sugests_alg = dar_sugestoes(10, sugestoes, 202266)
sugests_alg = []

In [8]:
i = 0
for item in sugests_alg:
    i += 1
    print(f'Periodo {i}\n')
    ch_periodo = 0
    for disc in item:
        ch_periodo += disciplinas[disc].ch
    print(f'Disciplinas: {item}, CHS: {ch_periodo/15}')


In [9]:
sugests_alg

[]

In [10]:
matriz = data_processing.get_matriz()

In [11]:
with open('legenda.csv', 'w', encoding='UTF-8') as f:
    for item in matriz.itertuples():
        f.write(f'{item[3]},{item[4]}\n\n')

## Testes

In [12]:
from data_processing import get_all_data_from_student, trim_user_data, get_course_list, user_based_suggestions, select_random_student, create_cache_user_arrays

In [13]:
user_id = select_random_student(data['formados'], 20, 2015)
user_data = get_all_data_from_student(user_id, data['formados'])

In [14]:
cache = create_cache_user_arrays(data["todos"], centralize_cosine = True)

### Calculamos precision e recall para os estudantes formados, removendo as últimas 12 disciplinas realizadas (aprox. 3 semestres)

In [15]:
import sys
sys.path.insert(0,'../utils')
from recommendation import recommend_dis, get_disciplinas_variables
_,_,df_844_optativas = get_disciplinas_variables()

../excel/dados_historicos_alunos_de_EngComputacaoCuritiba formados.csv
../excel/dados_historicos_alunos_de_EngComputacaoCuritiba regulares.csv


In [16]:
import pandas as pd
def trim_ate_optativa(num_optativas : int, dados_estudante : pd.DataFrame):
    achou = 0
    ind = 0

    df = dados_estudante.sort_values(['ANO','PERIODO.1'],axis=0)
    # Reset the index to add a new column with the default integer index
    df['RowNumber'] = range(1, len(df) + 1)
    df.set_index('RowNumber')

    for item in df.itertuples():
        if item[5] in list(df_844_optativas["CODIGO"]):
            print(item[5])
            print(disciplinas[item[5]].trilha)
            achou += 1
        if achou >= num_optativas:
            ind = item[0]
            ano = item[6]
            periodo = item[7]
            #print(f'{ano}  {periodo}')
            #print(ind)
            break

    if ind > 0:
        df = df[df['ANO'] <= ano]#int(ano.iloc[0])]
        if periodo == 1:
            df = df[~((df['ANO'] == ano) & (df['PERIODO.1'] == 2))]
        return df
    else:
        return df

In [17]:
from tqdm import tqdm
import concurrent.futures
from threading import Lock

class Log:
    path = ''
    mutex_lock = Lock()

    def clear(self):
        self.mutex_lock.acquire()
        with open(self.path, 'w') as f:
            f.write('')
        self.mutex_lock.release()


    def write(self, log):
        self.mutex_lock.acquire()
        with open(self.path, 'a') as f:
            f.write(log)
            f.write('\n')
        self.mutex_lock.release()

        

    def __init__(self, path):
        self.path = path

def teste_recomendacao(num_disciplinas_trim : int, cache):
    class Metrica:
        total = 0
        media = 0
        minimo = 0
        maximo = 0
        num_itens = 0

        def add_item(self, item : int):
            self.num_itens += 1
            if self.minimo > item:
                self.minimo = item
            if self.maximo < item:
                self.maximo = item
            self.total += item
            self.media = self.total / self.num_itens
        
        def print(self):
            print(f'Media: {self.media}\nMinimo: {self.minimo}\nMaximo: {self.maximo}\n')
        
    class Results:
        sugestoes = []
        sugestoes_alg = []
        estudante = 0
        precision = 0
        recall = 0
        true_positives = 0
        disciplinas_com_erro = []
        precision_ob = 0
        recall_ob = 0
        precision_op = 0
        recall_op = 0

        def __init__(self, sugestoes,sugestoes_alg,estudante,precision,recall,true_positives,disciplinas_com_erro,p_ob,r_ob,p_op,r_op):
            self.sugestoes = sugestoes
            self.sugestoes_alg = sugestoes_alg
            self.estudante = estudante
            self.precision = precision
            self.recall = recall
            self.true_positives = true_positives
            self.disciplinas_com_erro = disciplinas_com_erro
            self.precision_ob = p_ob
            self.recall_ob = r_ob
            self.precision_op = p_op
            self.recall_op = r_op
    

    log = Log('output/teste_rec.txt')
    log.clear()

    res_csv = Log('output/res.csv')
    res_csv.clear()

    def turn_to_date(x : str):
        try:
            x = int(x.split('/')[2][:2])
        except:
            #print(x)
            x = 0
        return x
            

    #data['formados']['INGRESSO'].map(turn_to_date)
    dados = data['formados'][data['formados']['INGRESSO'].map(turn_to_date) > 14]
    lista_estudantes = dados['ID_ANONIMO'].unique()
    #print(lista_estudantes)

    def thread_estudante(num_disciplinas_trim, estudante):
        #log.write(f'{i} started')
        #pbar.update(1)
        dados_estudante = get_all_data_from_student(int(estudante), data['formados'])
        if len(dados_estudante) == 0:
            print(dados_estudante)
            return 0
        dados_trimmed = trim_ate_optativa(dados_estudante, num_disciplinas_trim)
        if len(dados_trimmed.index) == 0:
            print(dados_trimmed)
            return 0
        # Sugerimos disciplinas com base nos dados tirando as ultimas num_disciplinas_trim disciplinas
        sugestoes = [key for key in dict(sorted(user_based_suggestions(estudante, data_rec=data['formados'], data_user=dados_trimmed,cache=cache, centralize_cosine=True, reduce_popularity_weight=False),key=lambda x:x[1], reverse=True))]
        #print(estudante)
        #print(sugestoes)
        if len(sugestoes) == 0:
            print(sugestoes)
            return 0
        
        discs_trimmed = list(trim_user_data(dados_estudante, num_disciplinas_trim)['CODIGO'].unique())
        sugestoes_alg = dar_sugestoes(20, sugestoes, estudante, dados_trimmed)
        disciplinas_alg = []
        for periodo in sugestoes_alg:
            for disc in periodo:
                disciplinas_alg.append(disc)
        if len(disciplinas_alg) == 0:
            print (estudante)
            return 0

        # Montar lista de equivalentes
        equivalentes_estudante = []
        disciplinas_com_erro = []
        lista_disc = [disc for disc in list(dados_estudante['CODIGO'].unique()) if disc not in discs_trimmed]
        for disc in lista_disc:
            try:
                for eq in disciplinas[disc].equivalentes:
                    equivalentes_estudante.append(eq)
            except KeyError:
                disciplinas_com_erro.append(disc)

        # Calcular precisao e recall
        true_positives = 0
        truep_ob = 0
        truep_op = 0
        len_alg_ob = 0
        len_list_ob = 0
        for disc in disciplinas_alg:
            if disc in lista_disc or disc in equivalentes_estudante:
                true_positives += 1
                if disc in disciplinas.keys() and disciplinas[disc].trilha == 'obrigatoria':
                    truep_ob += 1
                else:
                    truep_op += 1
            if disc in disciplinas.keys() and disciplinas[disc].trilha == 'obrigatoria':
                len_alg_ob += 1

        for disc in lista_disc:
            if disc in disciplinas.keys() and disciplinas[disc].trilha == 'obrigatoria':
                len_list_ob += 1
        
        result_precision=(true_positives/len(disciplinas_alg))
        result_recall=(true_positives/len(lista_disc))

        len_alg_op = len(disciplinas_alg) - len_alg_ob
        if len_alg_op == 0:
            len_alg_op = float('inf')
        if len_alg_ob == 0:
            len_alg_ob = float('inf')
        len_list_op = len(lista_disc) - len_alg_ob
        if len_list_op == 0:
            len_list_op = float('inf')
        if len_list_ob == 0:
            len_list_ob = float('inf')

        p_ob = truep_ob/len_alg_ob
        r_ob = truep_ob/len_list_ob

        p_op = truep_op/len_alg_op
        r_op = truep_op/len_list_op

        return Results(sugestoes,sugestoes_alg,estudante,result_recall,result_precision,true_positives, disciplinas_com_erro,p_ob,r_ob,p_op,r_op)
        
    
    pool = concurrent.futures.ThreadPoolExecutor(max_workers = 22)
    print(lista_estudantes)
    futures = [pool.submit(thread_estudante, num_disciplinas_trim,estudante) for estudante in lista_estudantes]

    print(f'waiting for {len(lista_estudantes)} threads')


    precision = Metrica()
    recall = Metrica()
    precision_ob = Metrica()
    recall_ob = Metrica()
    precision_op = Metrica()
    recall_op = Metrica()

    disciplinas_com_erro = []
    res_csv.write('ID_ANONIMO,PRECISION,RECALL,PRECISION_OB,RECALL_OB,PRECISION_OP,RECALL_OP')
    for i in tqdm(range(0,len(lista_estudantes))):
            item = futures[i]
            #item = lista_estudantes[i]
            result : Results = item.result() #thread_estudante(num_disciplinas_trim, item)
            if result != 0:
                precision.add_item(result.precision)
                recall.add_item(result.recall)
                precision_ob.add_item(result.precision_ob)
                recall_ob.add_item(result.recall_ob)
                precision_op.add_item(result.precision_op)
                recall_op.add_item(result.recall_op)
                log.write(f'\n{result.estudante} done\nTrue Positives: {result.true_positives}\nSugestoes: {result.sugestoes}\nSugestoes com pos proc: {result.sugestoes_alg}\nRecall: {result.recall}\nPrecision: {result.precision}\nRecall Obrigatorias: {result.recall_ob}\nPrecision Obrigatorias: {result.precision_ob}\nRecall Optativas: {result.recall_op}\nPrecision Optativas: {result.precision_op}\n')
                for disc in result.disciplinas_com_erro:
                    if disc not in disciplinas_com_erro:
                        disciplinas_com_erro.append(disc)
                res_csv.write(f'{int(result.estudante)},{result.precision},{result.recall},{result.precision_ob},{result.recall_ob},{result.precision_op},{result.recall_op}')
    #pool.shutdown(wait=True)
    print('Recall: \n')
    recall.print()
    print('Precision: \n')
    precision.print()
    print('Recall Obrigatorias: \n')
    recall_ob.print()
    print('Precision Obrigatorias: \n')
    precision_ob.print()
    print('Recall Optativas: \n')
    recall_op.print()
    print('Precision Optativas: \n')
    precision_op.print()




In [18]:
#teste_recomendacao(2,cache)

In [19]:
def turn_to_date(x : str):
    try:
        x = int(x.split('/')[2][:2])
    except:
        #print(x)
        x = 0
    return x

dados = data['formados'][data['formados']['INGRESSO'].map(turn_to_date) > 14]
lista_estudantes = dados['ID_ANONIMO'].unique()

for student in lista_estudantes:
    dados_estudante = dados[dados['ID_ANONIMO'] == student]
    dados_estudante.to_csv(f'input/dados_estudante/full/{int(student)}.csv')
    dados_estudante = trim_user_data(dados_estudante, 6)
    dados_estudante.to_csv(f'input/dados_estudante/trimmed/{int(student)}_trimmed_{6}.csv')


In [20]:
sugest = recommend_dis('input/dados_estudante/trimmed/150878_trimmed_2.csv','CliquesMaximais')
print(sugest)

CSV path: input/dados_estudante/trimmed/150878_trimmed_2.csv
Alg selecionado: CliquesMaximais
DiGraph with 55 nodes and 433 edges
[['FI70A', 1.0854503464203233], ['CSH30', 1.0277136258660509], ['FI70D', 0.9699769053117784], ['CSH42', 0.8545034642032333], ['GE70F', 0.6697459584295612], ['CSM43', 0.2655889145496536], ['CSM41', 0.2655889145496536], ['ES70B', 0.25404157043879905], ['CSH43', 0.21939953810623555], ['ED70U', 0.20785219399538105], ['ED70T', 0.19630484988452657], ['FI70B', 0.17321016166281755], ['CSR41', 0.15011547344110854], ['DI84D', 0.13856812933025406], ['CSM44', 0.12702078521939952], ['EL64B', 0.12702078521939952], ['CSB51', 0.11547344110854504], ['CSB53', 0.10392609699769052], ['FI70E', 0.10392609699769052], ['CSI53', 0.09237875288683603], ['CSM40', 0.09237875288683603], ['EL6AE', 0.09237875288683603], ['CSV40', 0.08083140877598152], ['CSR44', 0.08083140877598152], ['MA70C', 0.08083140877598152], ['CSG42', 0.08083140877598152], ['CSB41', 0.06928406466512703], ['CSV30', 0.

In [21]:
lista_sugestoes = [item[0] for item in sugest]
print(lista_sugestoes)

['FI70A', 'CSH30', 'FI70D', 'CSH42', 'GE70F', 'CSM43', 'CSM41', 'ES70B', 'CSH43', 'ED70U', 'ED70T', 'FI70B', 'CSR41', 'DI84D', 'CSM44', 'EL64B', 'CSB51', 'CSB53', 'FI70E', 'CSI53', 'CSM40', 'EL6AE', 'CSV40', 'CSR44', 'MA70C', 'CSG42', 'CSB41', 'CSV30', 'CSR43', 'CSR42', 'CSD41', 'CSG48', 'EL6CB', 'IF6BV', 'CSI58', 'CSI31', 'CSB54', 'CSI41', 'CSI54', 'CSR53', 'CSR47', 'FCH7HB', 'CSH44', 'CSV45', 'CSE40', 'CSM30', 'CSH41', 'CSD40', 'CSI56', 'CSG44']


In [22]:
print(df_844_optativas["CODIGO"])
print(list(df_844_optativas["CODIGO"]))

20721    CSI58
20726    EEY51
20727    CSB51
20738    EEL51
20752    CSB41
         ...  
49660    FI70D
49662    GE70F
49663    MA70C
49675    QB70F
49689    CSE47
Name: CODIGO, Length: 316, dtype: object
['CSI58', 'EEY51', 'CSB51', 'EEL51', 'CSB41', 'CSI53', 'CSB53', 'CSV30', 'CSV40', 'CSR41', 'CSR48', 'CSM40', 'ES70B', 'ES70J', 'ES70N', 'CSH30', 'CSH44', 'CSM45', 'CSM41', 'CSB54', 'CSI31', 'ES70J', 'ES70N', 'CSI53', 'CSB41', 'ED70T', 'CSM41', 'ES70B', 'ES70J', 'ES70N', 'CSH30', 'CSM43', 'CSH42', 'FI70D', 'FI70A', 'ED70U', 'CSH42', 'CSH30', 'CSB41', 'IF6BV', 'GE70F', 'CSM40', 'CSM43', 'CSH44', 'CSH30', 'MA70C', 'FI70A', 'EL64B', 'CSB53', 'CSB51', 'CSB51', 'CSI53', 'MA70C', 'FI70D', 'CSR42', 'EEY44', 'EEY41', 'EEY41', 'CSR41', 'EEY44', 'ED70T', 'CSV30', 'CSB53', 'CSM44', 'CSR41', 'CSB51', 'CSM44', 'CSB41', 'CSV30', 'ED70T', 'CSB54', 'CSV40', 'ED70U', 'ES70N', 'CSM44', 'CSR41', 'CSI53', 'ED70T', 'CSM44', 'CSM40', 'CSR44', 'CSV30', 'CSB54', 'CSV40', 'CSV30', 'ED70T', 'CSB41', 'CSA41', '

In [23]:
#trim_ate_optativa(1, get_all_data_from_student(182913, dados))
teste = get_all_data_from_student(150878,dados)
teste = trim_ate_optativa(9,teste)
print(teste)

ES70J
Optativas Isoladas
ES70N
Optativas Isoladas
ES70B
Optativas Isoladas
CSM41
Desenvolvimento Baseado em Plataformas Programacao
CSM45
Desenvolvimento Baseado em Plataformas Programacao
CSH30
IHC
CSH44
IHC
       ID_ANONIMO     CR  PERIODO  INGRESSO CODIGO   ANO  PERIODO.1  NOTA   CH SITUACAOALUNO  RowNumber
21856    150878.0  0.733       10  02/03/17  FI71M  2017          1   6.0   60       Formado          1
21860    150878.0  0.733       10  02/03/17  MA71A  2017          1   7.5   90       Formado          2
21867    150878.0  0.733       10  02/03/17  MA71B  2017          1   7.5   90       Formado          3
21878    150878.0  0.733       10  02/03/17  CSF13  2017          1   6.0   90       Formado          4
21903    150878.0  0.733       10  02/03/17  CSD20  2017          1   7.5   45       Formado          5
...           ...    ...      ...       ...    ...   ...        ...   ...  ...           ...        ...
21928    150878.0  0.733       10  02/03/17  CL74A  2022       

In [24]:
import sys,os
def teste_recomendacao_grafos(num_disciplinas_trim : int, alg : str):
    class Metrica:
        total = 0
        media = 0
        minimo = 0
        maximo = 0
        num_itens = 0

        def add_item(self, item : int):
            self.num_itens += 1
            if self.minimo > item:
                self.minimo = item
            if self.maximo < item:
                self.maximo = item
            self.total += item
            self.media = self.total / self.num_itens
        
        def print(self):
            print(f'Media: {self.media}\nMinimo: {self.minimo}\nMaximo: {self.maximo}\n')
        
    class Results:
        sugestoes = []
        sugestoes_alg = []
        estudante = 0
        precision = 0
        recall = 0
        true_positives = 0
        disciplinas_com_erro = []
        precision_ob = 0
        recall_ob = 0
        precision_op = 0
        recall_op = 0

        def __init__(self, sugestoes,sugestoes_alg,estudante,precision,recall,true_positives,disciplinas_com_erro,p_ob,r_ob,p_op,r_op):
            self.sugestoes = sugestoes
            self.sugestoes_alg = sugestoes_alg
            self.estudante = estudante
            self.precision = precision
            self.recall = recall
            self.true_positives = true_positives
            self.disciplinas_com_erro = disciplinas_com_erro
            self.precision_ob = p_ob
            self.recall_ob = r_ob
            self.precision_op = p_op
            self.recall_op = r_op
    

    log = Log('output/teste_rec.txt')
    log.clear()

    res_csv = Log('output/res.csv')
    res_csv.clear()

    def turn_to_date(x : str):
        try:
            x = int(x.split('/')[2][:2])
        except:
            #print(x)
            x = 0
        return x
            

    #data['formados']['INGRESSO'].map(turn_to_date)
    dados = data['formados'][data['formados']['INGRESSO'].map(turn_to_date) > 14]
    lista_estudantes = dados['ID_ANONIMO'].unique()
    for student in lista_estudantes:
        dados_estudante = dados[dados['ID_ANONIMO'] == student]
        dados_estudante.to_csv(f'input/dados_estudante/full/{int(student)}.csv')
        dados_estudante = trim_ate_optativa(num_disciplinas_trim, dados_estudante)
        dados_estudante.to_csv(f'input/dados_estudante/trimmed/{int(student)}_trimmed_{num_disciplinas_trim}.csv')
        #print(lista_estudantes)

    def thread_estudante(num_disciplinas_trim, estudante):
        #log.write(f'{i} started')
        #pbar.update(1)
        dados_estudante = get_all_data_from_student(int(estudante), data['formados'])
        if len(dados_estudante) == 0:
            print(dados_estudante)
            return 0
        dados_trimmed = trim_ate_optativa(num_disciplinas_trim,dados_estudante)
        if len(dados_trimmed.index) == 0:
            print(dados_trimmed)
            return 0
        # Sugerimos disciplinas com base nos dados tirando as ultimas num_disciplinas_trim disciplinas
        if alg != 'cosseno':
            sugestoes = [item[0] for item in recommend_dis(f'input/dados_estudante/trimmed/{int(estudante)}_trimmed_{num_disciplinas_trim}.csv', alg)]
        else:
            sugestoes = [key for key in dict(sorted(user_based_suggestions(estudante, data_rec=data['formados'], data_user=dados_trimmed,cache=cache, centralize_cosine=True, reduce_popularity_weight=False),key=lambda x:x[1], reverse=True))]
        #print(estudante)
        #print(sugestoes)
        if len(sugestoes) == 0:
            print(sugestoes)
            return 0
        
        discs_trimmed = list(dados_trimmed['CODIGO'].unique())
        sugestoes_alg = dar_sugestoes(20, sugestoes, estudante, dados_trimmed)
        disciplinas_alg = []
        for periodo in sugestoes_alg:
            for disc in periodo:
                disciplinas_alg.append(disc)
        if len(disciplinas_alg) == 0:
            print (estudante)
            return 0

        # Montar lista de equivalentes
        equivalentes_estudante = []
        disciplinas_com_erro = []
        lista_disc = [disc for disc in list(dados_estudante['CODIGO'].unique()) if disc not in discs_trimmed]
        for disc in lista_disc:
            try:
                for eq in disciplinas[disc].equivalentes:
                    equivalentes_estudante.append(eq)
            except KeyError:
                disciplinas_com_erro.append(disc)

        # Calcular precisao e recall
        true_positives = 0
        truep_ob = 0
        truep_op = 0
        len_alg_ob = 0
        len_list_ob = 0
        for disc in disciplinas_alg:
            if disc in lista_disc or disc in equivalentes_estudante:
                true_positives += 1
                if disc in disciplinas.keys() and disciplinas[disc].trilha == 'obrigatoria':
                    truep_ob += 1
                else:
                    truep_op += 1
            if disc in disciplinas.keys() and disciplinas[disc].trilha == 'obrigatoria':
                len_alg_ob += 1

        for disc in lista_disc:
            if disc in disciplinas.keys() and disciplinas[disc].trilha == 'obrigatoria':
                len_list_ob += 1
        
        if len(disciplinas_alg) == 0:
            len_disc_alg = float('inf')
        else:
            len_disc_alg = len(disciplinas_alg)
        result_precision=(true_positives/len_disc_alg)
        if len(lista_disc) == 0:
            len_lista_disc = float('inf')
        else:
            len_lista_disc = len(lista_disc)
        result_recall=(true_positives/len_lista_disc)

        len_alg_op = len(disciplinas_alg) - len_alg_ob
        if len_alg_op == 0:
            len_alg_op = float('inf')
        if len_alg_ob == 0:
            len_alg_ob = float('inf')
        len_list_op = len(lista_disc) - len_list_ob
        if len_list_op == 0:
            len_list_op = float('inf')
        if len_list_ob == 0:
            len_list_ob = float('inf')

        p_ob = truep_ob/len_alg_ob
        r_ob = truep_ob/len_list_ob

        p_op = truep_op/len_alg_op
        r_op = truep_op/len_list_op

        return Results(sugestoes,sugestoes_alg,estudante,result_recall,result_precision,true_positives, disciplinas_com_erro,p_ob,r_ob,p_op,r_op)
        
    
    #pool = concurrent.futures.ThreadPoolExecutor(max_workers = 22)
    print(lista_estudantes)
    #futures = [pool.submit(thread_estudante, num_disciplinas_trim,estudante) for estudante in lista_estudantes]

    print(f'waiting for {len(lista_estudantes)} threads')


    precision = Metrica()
    recall = Metrica()
    precision_ob = Metrica()
    recall_ob = Metrica()
    precision_op = Metrica()
    recall_op = Metrica()

    disciplinas_com_erro = []
    res_csv.write('ID_ANONIMO,PRECISION,RECALL,PRECISION_OB,RECALL_OB,PRECISION_OP,RECALL_OP')
    for i in tqdm(range(0,len(lista_estudantes))):
            #item = futures[i]
            item = lista_estudantes[i]
            #result : Results = item.result() 
            #sys.stdout = open(os.devnull, 'w')
            result : Results = thread_estudante(num_disciplinas_trim, item)
            #sys.stdout = sys.__stdout__
            if result != 0:
                precision.add_item(result.precision)
                recall.add_item(result.recall)
                precision_ob.add_item(result.precision_ob)
                recall_ob.add_item(result.recall_ob)
                precision_op.add_item(result.precision_op)
                recall_op.add_item(result.recall_op)
                log.write(f'\n{result.estudante} done\nTrue Positives: {result.true_positives}\nSugestoes: {result.sugestoes}\nSugestoes com pos proc: {result.sugestoes_alg}\nRecall: {result.recall}\nPrecision: {result.precision}\nRecall Obrigatorias: {result.recall_ob}\nPrecision Obrigatorias: {result.precision_ob}\nRecall Optativas: {result.recall_op}\nPrecision Optativas: {result.precision_op}\n')
                for disc in result.disciplinas_com_erro:
                    if disc not in disciplinas_com_erro:
                        disciplinas_com_erro.append(disc)
                res_csv.write(f'{int(result.estudante)},{result.precision},{result.recall},{result.precision_ob},{result.recall_ob},{result.precision_op},{result.recall_op}')
    #pool.shutdown(wait=True)
    print('Recall: \n')
    recall.print()
    print('Precision: \n')
    precision.print()
    print('Recall Obrigatorias: \n')
    recall_ob.print()
    print('Precision Obrigatorias: \n')
    precision_ob.print()
    print('Recall Optativas: \n')
    recall_op.print()
    print('Precision Optativas: \n')
    precision_op.print()



In [25]:
teste_recomendacao_grafos(2,'CliquesMaximais')

GE70F
Optativas Isoladas
FI70D
Fisica
CSH30
IHC
CSV30
Processamento Grafico
ES70N
Optativas Isoladas
CSH30
IHC
CSH30
IHC
CSV30
Processamento Grafico
ES70N
Optativas Isoladas
FI70A
Fisica
ES70N
Optativas Isoladas
CSH30
IHC
CSH30
IHC
CSB41
Banco De Dados
ES70N
Optativas Isoladas
GE70F
Optativas Isoladas
CSH30
IHC
EEY41
Engenharia Biomedica
CSH30
IHC
CSM44
Desenvolvimento Baseado em Plataformas Programacao
CSR44
Redes De Computadores
CSH30
IHC
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSR45
Redes De Computadores
ED70T
Optativas Isoladas
CSV30
Processamento Grafico
EEY51
Engenharia Biomedica
EEL51
Fisica
CSV30
Processamento Grafico
CSV40
Processamento Grafico
ES70J
Optativas Isoladas
ES70N
Optativas Isoladas
ED70T
Optativas Isoladas
CSB54
Banco De Dados
CSM43
Desenvolvimento Baseado em Plataformas Programacao
ES70J
Optativas Isoladas
FI70D
Fisica
FI70A
Fisica
CSM40
Desenvolvimento Baseado em Plataformas Programacao
CSM43
Desenvolvimento Baseado em Plataformas Programacao
ES70N
Op

  0%|          | 0/68 [00:00<?, ?it/s]

GE70F
Optativas Isoladas
FI70D
Fisica
CSV path: input/dados_estudante/trimmed/90729_trimmed_2.csv
Alg selecionado: CliquesMaximais
DiGraph with 57 nodes and 260 edges
CSH30
IHC
CSV30
Processamento Grafico
CSV path: input/dados_estudante/trimmed/118844_trimmed_2.csv
Alg selecionado: CliquesMaximais
DiGraph with 83 nodes and 949 edges
ES70N
Optativas Isoladas
CSH30
IHC
CSV path: input/dados_estudante/trimmed/123664_trimmed_2.csv
Alg selecionado: CliquesMaximais
DiGraph with 73 nodes and 608 edges


  4%|▍         | 3/68 [00:00<00:02, 25.86it/s]

CSH30
IHC
CSV30
Processamento Grafico
CSV path: input/dados_estudante/trimmed/123676_trimmed_2.csv
Alg selecionado: CliquesMaximais
DiGraph with 86 nodes and 915 edges
ES70N
Optativas Isoladas
FI70A
Fisica
CSV path: input/dados_estudante/trimmed/123677_trimmed_2.csv
Alg selecionado: CliquesMaximais
DiGraph with 74 nodes and 632 edges
ES70N
Optativas Isoladas
CSH30
IHC
CSV path: input/dados_estudante/trimmed/123686_trimmed_2.csv
Alg selecionado: CliquesMaximais


...

CSB54
Banco De Dados
CSB41
Banco De Dados
CSV path: input/dados_estudante/trimmed/190508_trimmed_2.csv
Alg selecionado: CliquesMaximais
DiGraph with 62 nodes and 421 edges
CSB53
Banco De Dados
CSB51
Banco De Dados
CSV path: input/dados_estudante/trimmed/190615_trimmed_2.csv
Alg selecionado: CliquesMaximais
DiGraph with 59 nodes and 396 edges
FI70A
Fisica
CSH30
IHC
CSV path: input/dados_estudante/trimmed/191284_trimmed_2.csv
Alg selecionado: CliquesMaximais


 97%|█████████▋| 66/68 [00:02<00:00, 24.96it/s]

DiGraph with 57 nodes and 382 edges
ES70J
Optativas Isoladas
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSV path: input/dados_estudante/trimmed/198372_trimmed_2.csv
Alg selecionado: CliquesMaximais
DiGraph with 60 nodes and 460 edges
CSH30
IHC
CSM44
Desenvolvimento Baseado em Plataformas Programacao
CSV path: input/dados_estudante/trimmed/198543_trimmed_2.csv
Alg selecionado: CliquesMaximais
DiGraph with 72 nodes and 544 edges
CSB54
Banco De Dados
EL75H
Engenharia De Software
CSV path: input/dados_estudante/trimmed/202261_trimmed_2.csv
Alg selecionado: CliquesMaximais
DiGraph with 61 nodes and 456 edges


100%|██████████| 68/68 [00:02<00:00, 24.70it/s]

Recall: 

Media: 0.6203744138301961
Minimo: 0
Maximo: 1.0

Precision: 

Media: 0.4407639277091242
Minimo: 0
Maximo: 0.72

Recall Obrigatorias: 

Media: 0.8024962576257804
Minimo: 0
Maximo: 1.0

Precision Obrigatorias: 

Media: 0.878449393488456
Minimo: 0
Maximo: 1.0

Recall Optativas: 

Media: 0.060860044775485955
Minimo: 0
Maximo: 0.3333333333333333

Precision Optativas: 

Media: 0.1557291666666667
Minimo: 0
Maximo: 1.0



In [26]:
import pandas as pd

In [27]:
teste = pd.read_csv('output/res.csv')
print(teste.describe())
print(teste.drop(teste[teste['RECALL_OP'] == 0].index).describe())

          ID_ANONIMO  PRECISION     RECALL  PRECISION_OB  RECALL_OB  PRECISION_OP  RECALL_OP
count      64.000000  64.000000  64.000000     64.000000  64.000000     64.000000  64.000000
mean   166778.843750   0.440764   0.620374      0.878449   0.802496      0.155729   0.060860
std     24296.722906   0.196079   0.272142      0.281204   0.253668      0.250858   0.087697
min     90729.000000   0.000000   0.000000      0.000000   0.000000      0.000000   0.000000
25%    150890.750000   0.305769   0.428571      1.000000   0.750000      0.000000   0.000000
50%    172207.500000   0.459361   0.683333      1.000000   0.908333      0.000000   0.000000
75%    183038.000000   0.600000   0.826087      1.000000   0.950000      0.250000   0.111111
max    202261.000000   0.720000   1.000000      1.000000   1.000000      1.000000   0.333333
          ID_ANONIMO  PRECISION     RECALL  PRECISION_OB  RECALL_OB  PRECISION_OP  RECALL_OP
count      25.000000  25.000000  25.000000     25.000000  25.000000   

In [28]:
for i in range(1,8):
    teste_recomendacao_grafos(i,'Comunidades')
    teste = pd.read_csv('output/res.csv')
    teste.describe().to_csv(f'output/desc_comunidades_{i}.txt')
    teste.drop(teste[teste['RECALL_OP'] == 0].index).describe().to_csv(f'output/desc_notnull_comunidades_{i}.txt')

GE70F
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
CSH30
IHC
CSR44
Redes De Computadores
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
ED70T
Optativas Isoladas
EEY51
Engenharia Biomedica
CSV30
Processamento Grafico
ES70J
Optativas Isoladas
ED70T
Optativas Isoladas
CSM43
Desenvolvimento Baseado em Plataformas Programacao
FI70D
Fisica
CSM40
Desenvolvimento Baseado em Plataformas Programacao
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSH30


  6%|▌         | 4/68 [00:00<00:01, 38.46it/s]

GE70F
Optativas Isoladas
CSV path: input/dados_estudante/trimmed/90729_trimmed_1.csv
Alg selecionado: Comunidades
DiGraph with 57 nodes and 260 edges
QQQQ
['CSI58' 'EEY51' 'CSB51' 'EEL51' 'CSB41' 'CSI53' 'CSB53' 'CSV30' 'CSV40'
 'CSR41' 'CSR48' 'CSM40' 'ES70B' 'ES70J' 'ES70N' 'CSH30' 'CSH44' 'CSM45'
 'CSM41' 'CSB54' 'CSI31' 'ED70T' 'CSM43' 'CSH42' 'FI70D' 'FI70A' 'ED70U'
 'IF6BV' 'GE70F' 'MA70C' 'EL64B' 'CSR42' 'EEY44' 'EEY41' 'CSM44' 'CSR44'
 'CSA41' 'CSA42' 'CSA45' 'EL75H' 'CSI41' 'CSE40' 'CSE43' 'CSD41' 'FI70B'
 'FI70E' 'QB74I' 'MA7AV' 'CSL41' 'EL6BB' 'CSR45' 'FCH7HC' 'CSE47' 'CSE48'
 'CSG42' 'CSR43' 'CSW45' 'CSB55' 'LEM702' 'CSW55' 'LEM701' 'QB70F']
ASSDFSAFASD
['ES70R', 'IF63O', 'EL64O', 'EL62O', 'MA63C', 'MA61A', 'FI61A', 'MA64A', 'MA65A', 'MA62A', 'MA63A', 'IF62J', 'FI63A', 'IF61C', 'GE60D', 'FI62B', 'FI64A', 'EL6AE', 'FI62A', 'MA61B', 'MA63B', 'EL65G', 'EL63B', 'IF63E', 'IF63C', 'IF63F', 'ES61A', 'QB62A', 'IF66J', 'EL66H', 'IF66C', 'IF65D', 'IF64J', 'EL65A', 'EL66C', 'EL68E', '

 19%|█▉        | 13/68 [00:00<00:01, 39.76it/s]

QQQQ
['CSI58' 'EEY51' 'CSB51' 'EEL51' 'CSB41' 'CSI53' 'CSB53' 'CSV30' 'CSV40'
 'CSR41' 'CSR48' 'CSM40' 'ES70B' 'ES70J' 'ES70N' 'CSH30' 'CSH44' 'CSM45'
 'CSM41' 'CSB54' 'CSI31' 'ED70T' 'CSM43' 'CSH42' 'FI70D' 'FI70A' 'ED70U'
 'IF6BV' 'GE70F' 'MA70C' 'EL64B' 'CSR42' 'EEY44' 'EEY41' 'CSM44' 'CSR44'
 'CSA41' 'CSA42' 'CSA45' 'EL75H' 'CSI41' 'CSE40' 'CSE43' 'CSD41' 'FI70B'
 'FI70E' 'QB74I' 'MA7AV' 'CSL41' 'EL6BB' 'CSR45' 'FCH7HC' 'CSE47' 'CSE48'
 'CSG42' 'CSR43' 'CSW45' 'CSB55' 'LEM702' 'CSW55' 'LEM701' 'QB70F']
ASSDFSAFASD
['ES70R', 'MA61B', 'MA61A', 'FI61A', 'FI71M', 'MA71A', 'MA71B', 'IF61B', 'ES61A', 'IF61C', 'QB70D', 'FI72N', 'MA62A', 'EL62O', 'FI72M', 'FI73A', 'MA72A', 'IF62C', 'QB62A', 'IF62J', 'FI63A', 'FI62A', 'IF63O', 'MA63B', 'MA63A', 'MA65A', 'ES70L', 'MA70G', 'MA70H', 'MA73A', 'ES60A', 'IF63E', 'EL63B', 'IF63C', 'FI74S', 'FI74N', 'EEX22', 'EEF11', 'EEX21', 'EEQ31', 'FI71S', 'EEB31', 'FI73S', 'ES70P', 'ES70G', 'QB70C', 'EEB22', 'FI72S', 'CSH30', 'EEB21', 'CSD20', 'CSE20', 'CSF20'

...

DiGraph with 76 nodes and 502 edges
QQQQ
['CSI58' 'EEY51' 'CSB51' 'EEL51' 'CSB41' 'CSI53' 'CSB53' 'CSV30' 'CSV40'
 'CSR41' 'CSR48' 'CSM40' 'ES70B' 'ES70J' 'ES70N' 'CSH30' 'CSH44' 'CSM45'
 'CSM41' 'CSB54' 'CSI31' 'ED70T' 'CSM43' 'CSH42' 'FI70D' 'FI70A' 'ED70U'
 'IF6BV' 'GE70F' 'MA70C' 'EL64B' 'CSR42' 'EEY44' 'EEY41' 'CSM44' 'CSR44'
 'CSA41' 'CSA42' 'CSA45' 'EL75H' 'CSI41' 'CSE40' 'CSE43' 'CSD41' 'FI70B'
 'FI70E' 'QB74I' 'MA7AV' 'CSL41' 'EL6BB' 'CSR45' 'FCH7HC' 'CSE47' 'CSE48'
 'CSG42' 'CSR43' 'CSW45' 'CSB55' 'LEM702' 'CSW55' 'LEM701' 'QB70F']
ASSDFSAFASD
['MA71A', 'MA71B', 'CSD20', 'CSF13', 'FI71M', 'FI71S', 'EEX11', 'MA72A', 'FI72M', 'QB70C', 'CSE20', 'CSF20', 'FI72N', 'FI72S', 'MA70G', 'MA70H', 'MA73A', 'FI73M', 'CSF30', 'FI73S', 'EEB21', 'EEX21', 'FI74S', 'FI74N', 'EEF11', 'FI74M', 'CSD21', 'EEB22', 'EEQ31', 'EEB31', 'EEE31', 'CSW30', 'ES70G', 'CSB30', 'CSA30', 'EEF21', 'CSG20', 'CSR31', 'CSE30', 'CSA31', 'CSO30', 'ED70T', 'EEE32', 'CSW40', 'CSX54', 'EEQ32', 'EEX22', 'CSW41', 'ES70P'

 97%|█████████▋| 66/68 [00:01<00:00, 40.31it/s]

ES70J
Optativas Isoladas
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSM40
Desenvolvimento Baseado em Plataformas Programacao
CSM43
Desenvolvimento Baseado em Plataformas Programacao
CSH30
IHC
CSM41
Desenvolvimento Baseado em Plataformas Programacao
CSH42
IHC
CSV path: input/dados_estudante/trimmed/198372_trimmed_7.csv
Alg selecionado: Comunidades
DiGraph with 74 nodes and 580 edges
QQQQ
['CSI58' 'EEY51' 'CSB51' 'EEL51' 'CSB41' 'CSI53' 'CSB53' 'CSV30' 'CSV40'
 'CSR41' 'CSR48' 'CSM40' 'ES70B' 'ES70J' 'ES70N' 'CSH30' 'CSH44' 'CSM45'
 'CSM41' 'CSB54' 'CSI31' 'ED70T' 'CSM43' 'CSH42' 'FI70D' 'FI70A' 'ED70U'
 'IF6BV' 'GE70F' 'MA70C' 'EL64B' 'CSR42' 'EEY44' 'EEY41' 'CSM44' 'CSR44'
 'CSA41' 'CSA42' 'CSA45' 'EL75H' 'CSI41' 'CSE40' 'CSE43' 'CSD41' 'FI70B'
 'FI70E' 'QB74I' 'MA7AV' 'CSL41' 'EL6BB' 'CSR45' 'FCH7HC' 'CSE47' 'CSE48'
 'CSG42' 'CSR43' 'CSW45' 'CSB55' 'LEM702' 'CSW55' 'LEM701' 'QB70F']
ASSDFSAFASD
['MA71A', 'MA71B', 'FI71S', 'CSD20', 'CSF13', 'FI71M', 'FI72N', 'FI72M', 'QB70C', 

100%|██████████| 68/68 [00:01<00:00, 38.34it/s]

QQQQ
['CSI58' 'EEY51' 'CSB51' 'EEL51' 'CSB41' 'CSI53' 'CSB53' 'CSV30' 'CSV40'
 'CSR41' 'CSR48' 'CSM40' 'ES70B' 'ES70J' 'ES70N' 'CSH30' 'CSH44' 'CSM45'
 'CSM41' 'CSB54' 'CSI31' 'ED70T' 'CSM43' 'CSH42' 'FI70D' 'FI70A' 'ED70U'
 'IF6BV' 'GE70F' 'MA70C' 'EL64B' 'CSR42' 'EEY44' 'EEY41' 'CSM44' 'CSR44'
 'CSA41' 'CSA42' 'CSA45' 'EL75H' 'CSI41' 'CSE40' 'CSE43' 'CSD41' 'FI70B'
 'FI70E' 'QB74I' 'MA7AV' 'CSL41' 'EL6BB' 'CSR45' 'FCH7HC' 'CSE47' 'CSE48'
 'CSG42' 'CSR43' 'CSW45' 'CSB55' 'LEM702' 'CSW55' 'LEM701' 'QB70F']
ASSDFSAFASD
['MA71B', 'MA71A', 'FI71M', 'CSF13', 'CSD20', 'FI71S', 'EEX11', 'MA72A', 'FI72M', 'QB70C', 'FI72N', 'CSE20', 'CSF20', 'FI72S', 'EEX21', 'MA70G', 'MA70H', 'MA73A', 'FI73M', 'CSF30', 'EEB21', 'FI73S', 'EEF11', 'FI74N', 'FI74S', 'FI74M', 'CSD21', 'EEB22', 'EEB31', 'EEQ31', 'EEE31', 'CSW30', 'ES70G', 'CSB30', 'CSA30', 'CSG20', 'EEF21', 'CSI30', 'CSR31', 'CSB54', 'EEX22', 'ELB23', 'CSO30', 'CSE30', 'EEE32', 'CSW40', 'CSW41', 'EEC21', 'ELTE3', 'ELF62', 'CSA31', 'CSW42', 'EEQ32'

In [29]:
for i in range(1,8):
    teste_recomendacao_grafos(i,'CliquesMaximais')
    teste = pd.read_csv('output/res.csv')
    teste.describe().to_csv(f'output/desc_cliquesmax_{i}.txt')
    teste.drop(teste[teste['RECALL_OP'] == 0].index).describe().to_csv(f'output/desc_notnull_cliquesmax_{i}.txt')

GE70F
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
CSH30
IHC
CSR44
Redes De Computadores
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
ED70T
Optativas Isoladas
EEY51
Engenharia Biomedica
CSV30
Processamento Grafico
ES70J
Optativas Isoladas
ED70T
Optativas Isoladas
CSM43
Desenvolvimento Baseado em Plataformas Programacao
FI70D
Fisica
CSM40
Desenvolvimento Baseado em Plataformas Programacao
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSH30


  0%|          | 0/68 [00:00<?, ?it/s]

GE70F
Optativas Isoladas
CSV path: input/dados_estudante/trimmed/90729_trimmed_1.csv
Alg selecionado: CliquesMaximais
DiGraph with 57 nodes and 260 edges
CSH30
IHC
CSV path: input/dados_estudante/trimmed/118844_trimmed_1.csv
Alg selecionado: CliquesMaximais


  4%|▍         | 3/68 [00:00<00:02, 26.89it/s]

DiGraph with 55 nodes and 570 edges
ES70N
Optativas Isoladas
CSV path: input/dados_estudante/trimmed/123664_trimmed_1.csv
Alg selecionado: CliquesMaximais
DiGraph with 42 nodes and 360 edges
CSH30
IHC
CSV path: input/dados_estudante/trimmed/123676_trimmed_1.csv
Alg selecionado: CliquesMaximais
DiGraph with 58 nodes and 552 edges
ES70N
Optativas Isoladas
CSV path: input/dados_estudante/trimmed/123677_trimmed_1.csv
Alg selecionado: CliquesMaximais
DiGraph with 42 nodes and 376 edges
ES70N
Optativas Isoladas
CSV path: input/dados_estudante/trimmed/123686_trimmed_1.csv
Alg selecionado: CliquesMaximais
DiGraph with 42 nodes and 360 edges


...

CSB54
Banco De Dados
CSV30
Processamento Grafico
CSB41
Banco De Dados
ED70T
Optativas Isoladas
CSV40
Processamento Grafico
CSV path: input/dados_estudante/trimmed/190495_trimmed_7.csv
Alg selecionado: CliquesMaximais
DiGraph with 75 nodes and 471 edges
190495.0
ED70T
Optativas Isoladas
ED70U
Optativas Isoladas
ES70N
Optativas Isoladas
CSR41
Redes De Computadores
CSI53
Sistemas Inteligentes
CSM44
Desenvolvimento Baseado em Plataformas Programacao
CSM40
Desenvolvimento Baseado em Plataformas Programacao
CSV path: input/dados_estudante/trimmed/190499_trimmed_7.csv
Alg selecionado: CliquesMaximais
DiGraph with 73 nodes and 535 edges
CSB54
Banco De Dados
CSB41
Banco De Dados
CSV30
Processamento Grafico
ED70T
Optativas Isoladas
CSV40
Processamento Grafico
CSV path: input/dados_estudante/trimmed/190508_trimmed_7.csv
Alg selecionado: CliquesMaximais
DiGraph with 75 nodes and 541 edges
190508.0


 97%|█████████▋| 66/68 [00:02<00:00, 28.28it/s]

CSB53
Banco De Dados
CSB51
Banco De Dados
CSV30
Processamento Grafico
CSA41
Algoritmos E Complexidade
CSA42
Algoritmos E Complexidade
CSA45
Algoritmos E Complexidade
CSV path: input/dados_estudante/trimmed/190615_trimmed_7.csv
Alg selecionado: CliquesMaximais
DiGraph with 70 nodes and 461 edges
190615.0
FI70A
Fisica
CSH30
IHC
CSH42
IHC
CSM45
Desenvolvimento Baseado em Plataformas Programacao
CSM44
Desenvolvimento Baseado em Plataformas Programacao
CSV path: input/dados_estudante/trimmed/191284_trimmed_7.csv
Alg selecionado: CliquesMaximais
DiGraph with 72 nodes and 494 edges
191284.0
ES70J
Optativas Isoladas
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSM40
Desenvolvimento Baseado em Plataformas Programacao
CSM43
Desenvolvimento Baseado em Plataformas Programacao
CSH30
IHC
CSM41
Desenvolvimento Baseado em Plataformas Programacao
CSH42
IHC
CSV path: input/dados_estudante/trimmed/198372_trimmed_7.csv
Alg selecionado: CliquesMaximais
DiGraph with 74 nodes and 580 edges
198372.0


100%|██████████| 68/68 [00:02<00:00, 27.36it/s]

CSH30
IHC
CSM44
Desenvolvimento Baseado em Plataformas Programacao
CSH42
IHC
CSM45
Desenvolvimento Baseado em Plataformas Programacao
CSM41
Desenvolvimento Baseado em Plataformas Programacao
CSV path: input/dados_estudante/trimmed/198543_trimmed_7.csv
Alg selecionado: CliquesMaximais
DiGraph with 76 nodes and 580 edges
CSB54
Banco De Dados
EL75H
Engenharia De Software
CSI53
Sistemas Inteligentes
CSI41
Sistemas Inteligentes
CSE40
Engenharia De Software
CSE43
Engenharia De Software
CSV path: input/dados_estudante/trimmed/202261_trimmed_7.csv
Alg selecionado: CliquesMaximais
DiGraph with 72 nodes and 475 edges
Recall: 

Media: 0.109375
Minimo: 0
Maximo: 1.0

Precision: 

Media: 0.047903138528138524
Minimo: 0
Maximo: 0.5

Recall Obrigatorias: 

Media: 0.10069444444444443
Minimo: 0
Maximo: 1.0

Precision Obrigatorias: 

Media: 0.13541666666666666
Minimo: 0
Maximo: 1.0

Recall Optativas: 

Media: 0.0078125
Minimo: 0
Maximo: 0.25

Precision Optativas: 

Media: 0.020833333333333332
Minimo: 0
M

In [30]:
for i in range(1,8):
    teste_recomendacao_grafos(i,'Kcliques')
    teste = pd.read_csv('output/res.csv')
    teste.describe().to_csv(f'output/desc_kcliques_{i}.txt')
    teste.drop(teste[teste['RECALL_OP'] == 0].index).describe().to_csv(f'output/desc_notnull_kcliques_{i}.txt')

GE70F
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
CSH30
IHC
CSR44
Redes De Computadores
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
ED70T
Optativas Isoladas
EEY51
Engenharia Biomedica
CSV30
Processamento Grafico
ES70J
Optativas Isoladas
ED70T
Optativas Isoladas
CSM43
Desenvolvimento Baseado em Plataformas Programacao
FI70D
Fisica
CSM40
Desenvolvimento Baseado em Plataformas Programacao
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSH30


  0%|          | 0/68 [00:00<?, ?it/s]

GE70F
Optativas Isoladas
CSV path: input/dados_estudante/trimmed/90729_trimmed_1.csv
Alg selecionado: Kcliques
DiGraph with 57 nodes and 260 edges


  1%|▏         | 1/68 [00:00<00:07,  8.86it/s]

CSH30
IHC
CSV path: input/dados_estudante/trimmed/118844_trimmed_1.csv
Alg selecionado: Kcliques
DiGraph with 55 nodes and 570 edges


...

FI70A
Fisica
CSH30
IHC
CSH42
IHC
CSM45
Desenvolvimento Baseado em Plataformas Programacao
CSM44
Desenvolvimento Baseado em Plataformas Programacao
CSV path: input/dados_estudante/trimmed/191284_trimmed_7.csv
Alg selecionado: Kcliques
DiGraph with 72 nodes and 494 edges
191284.0
ES70J
Optativas Isoladas
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSM40
Desenvolvimento Baseado em Plataformas Programacao
CSM43
Desenvolvimento Baseado em Plataformas Programacao
CSH30
IHC
CSM41
Desenvolvimento Baseado em Plataformas Programacao
CSH42
IHC
CSV path: input/dados_estudante/trimmed/198372_trimmed_7.csv
Alg selecionado: Kcliques
DiGraph with 74 nodes and 580 edges


 99%|█████████▊| 67/68 [00:07<00:00,  8.77it/s]

198372.0
CSH30
IHC
CSM44
Desenvolvimento Baseado em Plataformas Programacao
CSH42
IHC
CSM45
Desenvolvimento Baseado em Plataformas Programacao
CSM41
Desenvolvimento Baseado em Plataformas Programacao
CSV path: input/dados_estudante/trimmed/198543_trimmed_7.csv
Alg selecionado: Kcliques
DiGraph with 76 nodes and 580 edges
CSB54
Banco De Dados
EL75H
Engenharia De Software
CSI53
Sistemas Inteligentes
CSI41
Sistemas Inteligentes
CSE40
Engenharia De Software
CSE43
Engenharia De Software
CSV path: input/dados_estudante/trimmed/202261_trimmed_7.csv
Alg selecionado: Kcliques
DiGraph with 72 nodes and 475 edges


100%|██████████| 68/68 [00:07<00:00,  8.82it/s]

Recall: 

Media: 0.11666666666666667
Minimo: 0
Maximo: 1.0

Precision: 

Media: 0.05109668109668109
Minimo: 0
Maximo: 0.5

Recall Obrigatorias: 

Media: 0.1074074074074074
Minimo: 0
Maximo: 1.0

Precision Obrigatorias: 

Media: 0.14444444444444443
Minimo: 0
Maximo: 1.0

Recall Optativas: 

Media: 0.008333333333333333
Minimo: 0
Maximo: 0.25

Precision Optativas: 

Media: 0.022222222222222223
Minimo: 0
Maximo: 0.5



In [31]:
for i in range(1,8):
    teste_recomendacao_grafos(i,'cosseno')
    teste = pd.read_csv('output/res.csv')
    teste.describe().to_csv(f'output/desc_cosseno_{i}.txt')
    teste.drop(teste[teste['RECALL_OP'] == 0].index).describe().to_csv(f'output/desc_notnull_cosseno_{i}.txt')

GE70F
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
CSH30
IHC
CSR44
Redes De Computadores
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
ED70T
Optativas Isoladas
EEY51
Engenharia Biomedica
CSV30
Processamento Grafico
ES70J
Optativas Isoladas
ED70T
Optativas Isoladas
CSM43
Desenvolvimento Baseado em Plataformas Programacao
FI70D
Fisica
CSM40
Desenvolvimento Baseado em Plataformas Programacao
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSH30


  0%|          | 0/68 [00:00<?, ?it/s]

GE70F
Optativas Isoladas


  1%|▏         | 1/68 [00:01<01:13,  1.09s/it]

CSH30
IHC


...

198372.0
CSH30
IHC
CSM44
Desenvolvimento Baseado em Plataformas Programacao
CSH42
IHC
CSM45
Desenvolvimento Baseado em Plataformas Programacao
CSM41
Desenvolvimento Baseado em Plataformas Programacao


 99%|█████████▊| 67/68 [01:13<00:00,  1.03it/s]

CSB54
Banco De Dados
EL75H
Engenharia De Software
CSI53
Sistemas Inteligentes
CSI41
Sistemas Inteligentes
CSE40
Engenharia De Software
CSE43
Engenharia De Software


100%|██████████| 68/68 [01:14<00:00,  1.10s/it]

Recall: 

Media: 0.109375
Minimo: 0
Maximo: 1.0

Precision: 

Media: 0.047903138528138524
Minimo: 0
Maximo: 0.5

Recall Obrigatorias: 

Media: 0.10069444444444443
Minimo: 0
Maximo: 1.0

Precision Obrigatorias: 

Media: 0.13541666666666666
Minimo: 0
Maximo: 1.0

Recall Optativas: 

Media: 0.0078125
Minimo: 0
Maximo: 0.25

Precision Optativas: 

Media: 0.020833333333333332
Minimo: 0
Maximo: 0.5

